In [1]:
from modules_th.video_block import *
from modules_th.triplet_loss import *
from modules_th.inflator import *
from fastai.vision.all import *
from pathlib import Path
import torch.nn as nn
import torch
import time

KeyboardInterrupt: 

In [ ]:
class MyResizeTime(Transform):
    split_idx = None # 0- train 1- validation 
    def __init__(self, skip=2, l=50, drop_last=True,**kwargs): 
        self.skip = skip
        self.l = l
        self.drop_last = drop_last
        super().__init__(**kwargs)
        
    def encodes(self, vid:Video, split_idx=split_idx):
        '''create a list of frame-images (snippet) out a single video path'''
        l, skip, l_vid = self.l, self.skip, len(vid)
        if l_vid > l:
            idx = (len(vid)-l)//2 if split_idx else random.randint(0,len(vid)-l) # ** if validation always takes middle snip
            return vid[idx:idx+l]        
        else:
            vid = stretch(vid, l)         
        return vid
 

In [ ]:
path_charades = Path('/mnt/data/adrianlopez/Videos/Charades') 

using_df0 = False

n = 0 if using_df0 else ''
items = pd.read_csv(path_charades/f'df{n}.csv', index_col=0)

In [ ]:
get_paths = DF2Paths(path_charades, fps=24) if using_df0 else lambda df: df['paths']
get_lbls = lambda df: df['lbl']

vid_pl = [get_paths, Video.create, MyResizeTime(l=50), Resize(128), ToTensor(), IntToFloatTensor()]
lbl_pl = [get_lbls, Categorize()]
splits = RandomSplitter(seed=42)(items)

ds = Datasets(items, [vid_pl, lbl_pl], splits=splits)

In [ ]:
bs = 12
t = time.perf_counter()
for i, o in enumerate(ds.train):
    if i >= bs: break
time.perf_counter() - t

In [ ]:
n_el, n_lbl = 3, 4
bs = n_lbl * n_el
us = UniformizedShuffle(items['lbl'].iloc[splits[0]], n_el=n_el, n_lbl=n_lbl)

dls = ds.dataloaders(bs=bs, shuffle_fn=us, num_workers=6)

In [ ]:
learn = Learner(dls, 
                create3Dcnn(n_mid=200, n_out=len(dls.vocab)), 
                loss_func=MixedLoss(norm='euc', alpha=1), 
                splitter=my_splitter, 
                metrics=tl_accuracy)

In [ ]:
torch.cuda.empty_cache()

learn.fine_tune(8)
learn.export(path_charades/'trained_models')